In [ ]:
'''
Data is based on the transcation history on 2026-01-01

sliding window : 60 mins to test in-degree and out-degree diff and raw for finding spike nodes => no data is more than 3 or 5 threshold on in and out

test on pure window data with nan replaced by 0 => in > 1k has 14 nodes, out > 500 has 3 nodes
test on pure window data with nan kept as nan => in > 500 has 1, out> 500 has 3
volume zscore distribution to tell extreme transaction sizes

and daily test to test recurring transaction paths for finding frequently used nodes.

on daily data, no circular transaction path found.

'''

'\nData is based on the transcation history on 2026-01-01\n'

In [38]:
import networkx as nx
import pandas as pd
import math
#from plotnine import ggplot, aes, geom_line, labs, theme_bw,geom_col, element_text, theme,geom_histogram, scale_x_log10,scale_y_log10 , facet_wrap, element_rect,options,scale_color_manual
from plotnine import *
from collections import Counter

# File paths for input and output data
inputs = r"C:\Users\Shenghui\Documents\GitHub\MSc_DV_CC\Task3\blockchair_bitcoin_inputs_20260101.tsv"
outputs = r"C:\Users\Shenghui\Documents\GitHub\MSc_DV_CC\Task3\blockchair_bitcoin_outputs_20260101.tsv"

In [3]:
# Load data
idf = pd.read_csv(inputs,sep="\t")
odf = pd.read_csv(outputs,sep="\t")

In [4]:
# Display first 5 rows of input data
idf.head()

,block_id,transaction_hash,index,time,value,value_usd,recipient,type,script_hex,is_from_coinbase,...,spending_block_id,spending_transaction_hash,spending_index,spending_time,spending_value_usd,spending_sequence,spending_signature_hex,spending_witness,lifespan,cdd
0,925940,1c8fddb2190947d73d8abf570b6acbc0f71c14ddfa2726...,0,2025-12-01 00:04:53,226510,204.4117,1FGiZB7K757EUixGcyeyME6Jp8qQZEiUUk,pubkeyhash,76a9149c8ab044348d826b9ae88d698d575a45a6e8fc69...,0,...,930341,ed447d4fac95217c5ca6a18948b8fd0ded3422b4ff4d9c...,0,2026-01-01 00:06:10,198.2325,4294967295,47304402206e8cb8c09e63ee1581f27a29296f35be5aa2...,NaN,2678477,0.070220
1,925940,edb174b0c2b5d50cd4204991fe98eae0653eec81c24ccc...,0,2025-12-01 00:04:53,161927,146.1294,1FGiZB7K757EUixGcyeyME6Jp8qQZEiUUk,pubkeyhash,76a9149c8ab044348d826b9ae88d698d575a45a6e8fc69...,0,...,930341,ed447d4fac95217c5ca6a18948b8fd0ded3422b4ff4d9c...,1,2026-01-01 00:06:10,141.7120,4294967295,4730440220095a6339ff14bb4e00f182df7ae915947c7e...,NaN,2678477,0.050199
2,925945,7d60e781c169bd564a848554ff62b6145fe4b81ae5783b...,0,2025-12-01 01:22:43,231716,209.1098,1GzgDCeDyeb4sKQZJZu5rUjAcgRvBBJiY7,pubkeyhash,76a914af7253e68afa9b6bde29f15829b27fd9c3f6dd21...,0,...,930341,ea9e624f3d62e5b9c3d685b6183d96ae9d263675e77440...,0,2026-01-01 00:06:10,202.7886,4294967295,473044022073c2312d24758ca5a592ea37d20883d27aa5...,NaN,2673807,0.071709
3,925943,125798cb5ea3af6e342561c2a670f3d70159b26b063549...,0,2025-12-01 00:42:53,152700,137.8026,1GzgDCeDyeb4sKQZJZu5rUjAcgRvBBJiY7,pubkeyhash,76a914af7253e68afa9b6bde29f15829b27fd9c3f6dd21...,0,...,930341,ea9e624f3d62e5b9c3d685b6183d96ae9d263675e77440...,1,2026-01-01 00:06:10,133.6369,4294967295,47304402206eeb5a1820a8c7305bab42d3e580e043ec15...,NaN,2676197,0.047298
4,925945,67cbe3c8c0a185fca6626cc1980a32912f7e58e8c03890...,0,2025-12-01 01:22:43,231716,209.1098,1GzgDCeDyeb4sKQZJZu5rUjAcgRvBBJiY7,pubkeyhash,76a914af7253e68afa9b6bde29f15829b27fd9c3f6dd21...,0,...,930341,809e1a85a322210d6e1fdf581c57d121e7afc3818ca176...,0,2026-01-01 00:06:10,202.7886,4294967295,47304402202260e21c338a3894b2c7ff19bb21edffb7bf...,NaN,2673807,0.071709


In [5]:
# Display data types of input dataframe
idf.dtypes

block_id                       int64
transaction_hash              object
index                          int64
time                          object
value                          int64
value_usd                    float64
recipient                     object
type                          object
script_hex                    object
is_from_coinbase               int64
is_spendable                   int64
spending_block_id              int64
spending_transaction_hash     object
spending_index                 int64
spending_time                 object
spending_value_usd           float64
spending_sequence              int64
spending_signature_hex        object
spending_witness              object
lifespan                       int64
cdd                          float64
dtype: object

In [6]:
# Display first 5 rows of output data
odf.head()

,block_id,transaction_hash,index,time,value,value_usd,recipient,type,script_hex,is_from_coinbase,is_spendable
0,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,0,2026-01-01 00:06:10,546,0.4778,3Eif1JfqeMERRsQHtvGEacNN9hhuvnsfe9,scripthash,a9148ee90177614ecde53314fd67c46162f315852a0787,1,-1
1,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,1,2026-01-01 00:06:10,314440653,275185.8750,3Awm3FNpmwrbvAFVThRUFqgpbVuqWisni9,scripthash,a9146582f2551e2a47e1ae8b03fb666401ed7c4552ef87,1,-1
2,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,1,2026-01-01 00:06:10,314440653,275185.8750,3Awm3FNpmwrbvAFVThRUFqgpbVuqWisni9,scripthash,a9146582f2551e2a47e1ae8b03fb666401ed7c4552ef87,1,-1
3,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,1,2026-01-01 00:06:10,314440653,275185.8750,3Awm3FNpmwrbvAFVThRUFqgpbVuqWisni9,scripthash,a9146582f2551e2a47e1ae8b03fb666401ed7c4552ef87,1,-1
4,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,2,2026-01-01 00:06:10,0,0.0000,d-bb48d5ef5dd5147a3c28f32e4055160c,nulldata,6a24aa21a9edd65a5527e7787b1a7d73682aed9e2d8d9d...,1,0


In [7]:
# Display data types of output dataframe
odf.dtypes

block_id              int64
transaction_hash     object
index                 int64
time                 object
value                 int64
value_usd           float64
recipient            object
type                 object
script_hex           object
is_from_coinbase      int64
is_spendable          int64
dtype: object

In [8]:
# Count occurrences of each recipient in input and output dataframes (address)
icounts = idf["recipient"].value_counts()
print(icounts)

ocounts = odf["recipient"].value_counts()
print(ocounts)

recipient
bc1pvad59hdlahwa66yv6qdn8vgd5d7p4usx88x54u0gqzz3y85yrcpq0fcfv2    46136
bc1pv6eqwxkxxda5nj49cry8z7lqnc2ft5hxz3jf0mx3v67e4c0krc8qzgjaa7    33957
bc1pwucn9l7vhfcp4k93xs5nu4r7tmd7qqmcnz5ekl877fp0l6cky7rqjgz4ck    29036
bc1pcecas3up00urhzjpqf5yagndfs7mhfmzuyl4yk68neqzgnsxjh9s7vk4ww    21451
bc1pxpyjtcmr8kuct4rtpfjjm4shc05zm08r0mzz5986tn5q5sulm58qq3zlam    19585
                                                                  ...  
13FnJDGha3aPD8e86T4CDhCyNf6KJ8jxuC                                    1
bc1qvf6jdkzcp27gytq57rxkqvhqanr8zgfmusn445                            1
bc1qr6l0wyx05qcm44gg9s9n66pz2ac76ftrnkk65q                            1
bc1qmkwdj2wcw6dkwa6e6dj957lpfze4f23lvck282                            1
bc1q74ex69ygl07fa7t9zm5xe09ym8e7uym5wy7gdmspqsr7fgryp7gsufpqhl        1
Name: count, Length: 400426, dtype: int64
recipient
d-39fb509d8eb10633e0e951c77bd3a7cb                                140748
bc1pvad59hdlahwa66yv6qdn8vgd5d7p4usx88x54u0gqzz3y85yrcpq0fcfv2     47860


In [136]:
# Merge input and output dataframes on transaction_hash and index
df = pd.merge(
    odf,
    idf,
    on=['transaction_hash','index'], # Merge on transaction_hash and index to link created (output) and spent (input) outputs
    how='left',
    suffixes=('_created','_spent') # Suffixes to distinguish created (output) and spent (input) columns
)
#df.to_csv("./Task3/merged_transaction_data_20260101.csv", index=False)

In [63]:
# Display first 5 rows of merged data
df.head()

,block_id_created,transaction_hash,index,time_created,value_created,value_usd_created,recipient_created,type_created,script_hex_created,is_from_coinbase_created,...,spending_block_id,spending_transaction_hash,spending_index,spending_time,spending_value_usd,spending_sequence,spending_signature_hex,spending_witness,lifespan,cdd
0,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,0,2026-01-01 00:06:10,546,0.4778,3Eif1JfqeMERRsQHtvGEacNN9hhuvnsfe9,scripthash,a9148ee90177614ecde53314fd67c46162f315852a0787,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,1,2026-01-01 00:06:10,314440653,275185.8750,3Awm3FNpmwrbvAFVThRUFqgpbVuqWisni9,scripthash,a9146582f2551e2a47e1ae8b03fb666401ed7c4552ef87,1,...,930442.0,c42bd5bb6c19218286f43589739190e541a6f850b146cb...,13.0,2026-01-01 15:32:47,275185.875,0.0,16001492858fe8411464453831172e18184f0b79a82924,304402200d8b7614b6db4c36164170b0654e75dfff78c9...,55597.0,2.023375
2,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,1,2026-01-01 00:06:10,314440653,275185.8750,3Awm3FNpmwrbvAFVThRUFqgpbVuqWisni9,scripthash,a9146582f2551e2a47e1ae8b03fb666401ed7c4552ef87,1,...,930442.0,c42bd5bb6c19218286f43589739190e541a6f850b146cb...,13.0,2026-01-01 15:32:47,275185.875,0.0,16001492858fe8411464453831172e18184f0b79a82924,304402200d8b7614b6db4c36164170b0654e75dfff78c9...,55597.0,2.023375
3,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,1,2026-01-01 00:06:10,314440653,275185.8750,3Awm3FNpmwrbvAFVThRUFqgpbVuqWisni9,scripthash,a9146582f2551e2a47e1ae8b03fb666401ed7c4552ef87,1,...,930442.0,c42bd5bb6c19218286f43589739190e541a6f850b146cb...,13.0,2026-01-01 15:32:47,275185.875,0.0,16001492858fe8411464453831172e18184f0b79a82924,304402200d8b7614b6db4c36164170b0654e75dfff78c9...,55597.0,2.023375
4,930341,1095a28fd257ecf76947002121e233d780a33d73bd9fdb...,2,2026-01-01 00:06:10,0,0.0000,d-bb48d5ef5dd5147a3c28f32e4055160c,nulldata,6a24aa21a9edd65a5527e7787b1a7d73682aed9e2d8d9d...,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
# Get the number of rows in the merged dataframe
len(df)

1524592

In [137]:
# Remove rows with missing spending_transaction_hash (the UTXOs that have not been spent yet)
df = df.dropna(subset=['spending_transaction_hash'])
len(df)
# convert spending_time to datetime format
df['spending_time'] = pd.to_datetime(df['spending_time'])

In [13]:
# Display columns of the merged dataframe
df.columns

Index(['block_id_created', 'transaction_hash', 'index', 'time_created',
       'value_created', 'value_usd_created', 'recipient_created',
       'type_created', 'script_hex_created', 'is_from_coinbase_created',
       'is_spendable_created', 'block_id_spent', 'time_spent', 'value_spent',
       'value_usd_spent', 'recipient_spent', 'type_spent', 'script_hex_spent',
       'is_from_coinbase_spent', 'is_spendable_spent', 'spending_block_id',
       'spending_transaction_hash', 'spending_index', 'spending_time',
       'spending_value_usd', 'spending_sequence', 'spending_signature_hex',
       'spending_witness', 'lifespan', 'cdd'],
      dtype='object')

In [ ]:
class BarChartDetail:
    '''Class to hold details for bar chart creation, and used in BitcoinTran'''
    def __init__(self,x_col_name:str, y_col_name:str, xlabel:str, ylabel:str, title:str, path:str) -> None:
        self.x_col = x_col_name
        self.y_col = y_col_name
        self.xlabel = xlabel
        self.ylabel = ylabel
        self.title = title
        self.path = path
    
class BitcoinTran:
    '''Class to process bitcoin transaction data and create network graphs and statistics'''
    def __init__(self,st:pd.Timestamp,et:pd.Timestamp,df:pd.DataFrame) -> None:
        self.st = st
        self.et = et
        self.df = df
        self.df_window = None

    def set_df_window_mins(self,deltaMins:int) -> 'BitcoinTran': # set the dataframe for a time window of deltaMins minutes
        self.ed = self.st + pd.Timedelta(minutes=deltaMins)
        self.df_window = self.df[(self.df['spending_time'] >= self.st) & (self.df['spending_time'] < self.ed)]
        self.network_path = f'./Task3/network/{self.st.strftime("%H%M")}-{self.ed.strftime("%H%M")}_network.gexf'
        print("number of rows for this window: ", len(self.df_window))
        return self
    
    def set_df_window_fromto(self,st:pd.Timestamp,et:pd.Timestamp) -> 'BitcoinTran': # set the dataframe for a time window from st to et
        self.df_window = self.df[(self.df['spending_time'] >= st) & (self.df['spending_time'] < et)]
        self.network_path = f'./Task3/network/{st.strftime("%H%M")}-{et.strftime("%H%M")}_network.gexf'
        print("number of rows for this window: ", len(self.df_window))
        return self
    
    def process_data(self,source:str,target:str,edge_attr:list) -> 'BitcoinTran': # create the networkx graph from a pandas dataframe
        self.G = nx.from_pandas_edgelist(
                self.df_window,
                source=source,
                target=target,
                edge_attr=edge_attr,
                create_using=nx.MultiDiGraph() # directed graph with multiple edges allowed
            )
        return self

    def create_stats_graph(self,sourceDetail:BarChartDetail,targetDetail:BarChartDetail,edgeDetail:BarChartDetail,volumeDetail:BarChartDetail) -> 'BitcoinTran': # create statistics graphs
        in_deg = pd.DataFrame(self.G.in_degree(), columns=['node', 'in_degree']) # in-degree dataframe
        out_deg = pd.DataFrame(self.G.out_degree(), columns=['node', 'out_degree']) # out-degree dataframe
        # Count multiple edges between same source and target
        edge_counts = Counter((u, v) for u, v, k in self.G.edges(keys=True))
        df_edges = pd.DataFrame([
                        {'source': u, 'target': v, 'counts': c}
                        for (u, v), c in edge_counts.items()
                    ])
        # Save raw dataframes befrore filtering
        in_deg.to_csv(f"{sourceDetail.path}_raw.csv", index=False)
        print(f"Raw in-degree dataframe is written to {sourceDetail.path}_raw.csv")
        out_deg.to_csv(f"{targetDetail.path}_raw.csv", index=False)
        print(f"Raw out-degree dataframe is written to {targetDetail.path}_raw.csv")
        df_edges.to_csv(f"{edgeDetail.path}_raw.csv", index=False)
        print(f"Raw edge counts dataframe is written to {edgeDetail.path}_raw.csv")
        
        # Filter all with counts >= 2
        df_edges = df_edges[df_edges['counts'] >= 2]
        multi_source = in_deg[in_deg['in_degree'] >= 2]
        multi_target = out_deg[out_deg['out_degree'] >= 2]
        # Create bar charts for multi-source
        self.create_bar_chart(multi_source, sourceDetail.y_col,
                              sourceDetail.xlabel, sourceDetail.ylabel,
                              sourceDetail.title, sourceDetail.path)
        # Create bar charts for multi-target              
        self.create_bar_chart(multi_target, targetDetail.y_col,
                              targetDetail.xlabel, targetDetail.ylabel,
                              targetDetail.title, targetDetail.path)
        # Create bar charts for multi-edges
        self.create_bar_chart(df_edges, edgeDetail.y_col,
                              edgeDetail.xlabel, edgeDetail.ylabel,
                              edgeDetail.title, edgeDetail.path)
        # Create volume histogram
        self.create_volume_histogram(volumeDetail.x_col, 
                                     volumeDetail.xlabel, volumeDetail.ylabel,
                                     volumeDetail.title, volumeDetail.path)
        return self
    
    def create_network_graph(self) -> 'BitcoinTran': # save the networkx graph to a gexf file
        nx.write_gexf(self.G,self.network_path)    
        print(f"network file is created at {self.network_path}")
        return self


    def create_bar_chart(self, counts_df:pd.DataFrame, x:str, xlabel:str, ylabel:str, title:str, filename:str="./barchart") -> None: # create a bar chart with chunking for both samll and large datasets
        # Save the raw counts dataframe to csv
        counts_df.to_csv(f"{filename}.csv", index=False)
        print(f"Raw dataframe is written to {filename}.csv")
        # Aggregation & Sorting
        hist_df = counts_df[x].value_counts().reset_index()
        hist_df.columns = [x, 'number']
        hist_df = hist_df.sort_values(by=x)
        # Create Subplot Groups (Chunking)
        limit =100
        hist_df['subplot_group'] = [i // limit for i in range(len(hist_df))]
        #print(hist_df)
        # Create a nice label for the subplot headers (e.g., "Batch 1", "Batch 2")
        hist_df['group_label'] = "Batch " + (hist_df['subplot_group'] + 1).astype(str)
        n_subplots = hist_df['subplot_group'].max() + 1              # Total number of subplots needed
        
        # Calculate width based on the "fullest" subplot (usually 100)
        max_bars_in_subplot = min(len(hist_df), limit)
        width = max(10, max_bars_in_subplot * 0.15) 
        
        # Height needs to grow: Base height (4) * Number of subplots
        height = 4 * n_subplots 

        # Plotting
        p = (
            ggplot(hist_df, aes(x=x, y='number')) +
            geom_col(fill="steelblue") +
            labs(x=xlabel, y=ylabel, title=title) +
            scale_y_log10() + # Log scale for y-axis
            theme_bw() + 
            theme(
                figure_size=(width, height),
                axis_text_x=element_text(rotation=0, ha='center'),
                plot_title=element_text(size=13, hjust=0.5),
                axis_title=element_text(size=11),
                # Style the headers for each subplot
                strip_background=element_rect(fill="#e0e0e0"),
                strip_text=element_text(size=11, face="bold") 
            )
        )
        # More than 1 chunk => create subplots
        if n_subplots > 1:
            # scales='free_x': Each subplot gets its own independent x-axis & ncol=1: Stack them vertically
            p = p + facet_wrap('~group_label', scales='free_x', ncol=1)
        
        p.save(filename, dpi=300, limitsize=False)
        print(f"Bar chart saved to {filename}")

    def create_volume_histogram(self,amount_col:str,xlabel:str,ylabel:str,title:str,filename:str="./vol_hist") -> None: # create volume histogram
        # Save transaction data to csv
        cols_to_save = ['transaction_hash', 'spending_transaction_hash']
        existing_cols = [c for c in cols_to_save if c in self.df_window.columns]
        if amount_col not in existing_cols and amount_col in self.df_window.columns:
            existing_cols.append(amount_col)
        self.df_window[existing_cols].sort_values(by=amount_col,ascending=False, kind="mergesort").to_csv(f"{filename}.csv", index=False)
        print(f"Transaction data saved to {filename}.csv")
        # Plotting
        p = (
            ggplot(self.df_window, aes(x=amount_col)) + 
            geom_histogram(bins=100, fill="orange", alpha=0.8) +
            scale_x_log10() +  scale_y_log10() +
            labs(x=xlabel, y=ylabel, title=title) +
            theme_bw() +
            theme(
                    axis_text_x=element_text(rotation=0, ha='center'),
                    legend_text=element_text(size=8),
                    legend_title=element_text(size=9),
                    axis_title=element_text(size=11),
                    plot_title=element_text(size=13,hjust=0.5)
                )
        )
        p.save(filename, dpi=300,limitsize=False)
        print(f"Volume histogram saved to {filename}.png")
        

In [139]:
# Set parameters
start_time = pd.to_datetime('2026-01-01 00:00:00')
end_time = pd.to_datetime('2026-01-02 00:00:00')
 # Networkx graph parameters using transaction hash as nodes to see the cash flow
source='transaction_hash'
target='spending_transaction_hash'
edge_attr='value_spent'  # unit in satoshi
# Bar chart labels
xlabel_out = "Number of Targets from a Transaction hash"
xlabel_in = "Number of Sources from a Trasaction hash"
xlabel_par = "Number of repeated transaction path"

ylabel = "Counts - Log scale"

In [140]:
#  Declare BitcoinTran object
B = BitcoinTran(start_time,end_time,df)

In [19]:
# Titles for 10-minute windows
title_out = "Frequency of Multi-Target Transactions Between 00:00 to 00:10 on 2026-01-01"
title_in = "Frequency of Multi-Source Transactions Between 00:00 to 00:10 on 2026-01-01"
title_par = "Recurring Transaction Path Frequency Between 00:00 to 00:10 on 2026-01-01"
title_vol = "Distribution of Transaction Sizes \nBetween 00:00 to 00:10 on 2026-01-01"

# Create BitcoinTran object and process data for 10-minute window
target10  = BarChartDetail('node','out_degree',xlabel_out,ylabel,
                title_out,"./Task3/10mins/10mins_out")

source10 = BarChartDetail('node','in_degree',xlabel_in,ylabel,
                        title_in,"./Task3/10mins/10mins_in")

edge10 = BarChartDetail('source','counts',xlabel_par,ylabel,
                          title_par, "./Task3/10mins/10mins_par")

vol10 = BarChartDetail(edge_attr,'', 'Trans value - Log scale', 'Count - Log scale',
                       title_vol, "./Task3/10mins/10mins_vol")

In [20]:
# Create stats graphs for 10-minute window
b10 = B.set_df_window_mins(10).process_data(source,target,edge_attr).create_stats_graph(source10,target10,edge10,vol10)

number of rows for this window:  1906
Raw in-degree dataframe is written to ./Task3/10mins/10mins_in_raw.csv
Raw out-degree dataframe is written to ./Task3/10mins/10mins_out_raw.csv
Raw edge counts dataframe is written to ./Task3/10mins/10mins_par_raw.csv
Raw dataframe is written to ./Task3/10mins/10mins_in.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 10 x 4 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/10mins/10mins_in
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 10 x 4 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/10mins/10mins_out
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Bar chart saved to ./Task3/10mins/10mins_in
Raw dataframe is written to ./Task3/10mins/10mins_out.csv
Bar chart saved to ./Task3/10mins/10mins_out
Raw dataframe is written to ./Task3/10mins/10mins_par.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 10 x 4 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/10mins/10mins_par
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/10mins/10mins_vol


Bar chart saved to ./Task3/10mins/10mins_par
Transaction data saved to ./Task3/10mins/10mins_vol.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Volume histogram saved to ./Task3/10mins/10mins_vol.png


In [30]:
# open the created file in Gephi
# Select ForceAtlas 2 as layout & tick Prevent Overlap and LinLog mode
# In Apperance:
#      Each node will be assigned a colour. A node will be more red if it has more sources of income.
#      Nodes tab:  
#             Select paint icon => Click Ranking => select In-Dgree => select rainbow => select invert
#      Each node has different size. A node will be bigger if it has more targets.
#      Nodes tab:
#             Select node_size icon => Click Ranking => select Out-Dgree => select min_size = 2 & max_size = 10
#      Each edge will be assigned a colour. An edge will be more red if it is higher value in $.
#      Edges tab:
#             Select paint icon => Click Ranking => select value_spent => select rainbow => select invert 
# Since nx.MultiDiGraph() is used, the parallel edges will be combined and thicken the edge between two nodes. 
b10.create_network_graph()

network file is created at ./Task3/network/0000-0010_network.gexf


In [141]:
# Plot the overall stats for all transaction on 2026-01-01
# Dont generate the gexf file, it wont be able to open by Gephi, too many nodes and edges to rendar.
title_out = "Frequency of Multi-Target Transactions Between 00:00 to 24:00 on 2026-01-01"
title_in = "Frequency of Multi-Source Transactions Between 00:00 to 24:00 on 2026-01-01"
title_par = "Recurring Transaction Path Frequency Between 00:00 to 24:00 on 2026-01-01"
title_vol = "Distribution of Transaction Sizes Between 00:00 to 24:00 on 2026-01-01"

targetday = BarChartDetail('node','out_degree',xlabel_out,ylabel,
                title_out,"./Task3/day/daymins_out")

sourceday = BarChartDetail('node','in_degree',xlabel_in,ylabel,
                         title_in,"./Task3/day/daymins_in")

xlabel_par = "Number of recurring transaction path (Source -> Target)"

edgeday = BarChartDetail('source','counts',xlabel_par,ylabel,
                          title_par, "./Task3/day/daymins_par")

volday = BarChartDetail(edge_attr,'', 'Trans value - Log scale', ylabel,
                       title_vol, "./Task3/day/daymins_vol")

In [142]:
# generate the stats for daily, save it to ./Task3/day/
bday = B.set_df_window_mins(60*24).process_data(source,target,edge_attr).create_stats_graph(sourceday,targetday,edgeday,volday)

number of rows for this window:  766760
Raw in-degree dataframe is written to ./Task3/day/daymins_in_raw.csv
Raw out-degree dataframe is written to ./Task3/day/daymins_out_raw.csv
Raw edge counts dataframe is written to ./Task3/day/daymins_par_raw.csv
Raw dataframe is written to ./Task3/day/daymins_in.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 15.0 x 16 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/day/daymins_in
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Bar chart saved to ./Task3/day/daymins_in
Raw dataframe is written to ./Task3/day/daymins_out.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 15.0 x 12 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/day/daymins_out
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Bar chart saved to ./Task3/day/daymins_out
Raw dataframe is written to ./Task3/day/daymins_par.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 10 x 4 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/day/daymins_par
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Bar chart saved to ./Task3/day/daymins_par
Transaction data saved to ./Task3/day/daymins_vol.csv


c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 20 x 10 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/day/daymins_vol
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


Volume histogram saved to ./Task3/day/daymins_vol.png


In [72]:
# Find simple cycles in the daily graph
bday_G = bday.G
cycles = list(nx.simple_cycles(bday_G))
print(cycles)     # print all cycles              
#short_cycles = [c for c in cycles if len(c) <= 5]  # 1-3 hops
#print("Suspicious short cycles:", short_cycles)

[]


In [24]:
# Titles for 60-minute windows
title_out = "Frequency of Multi-Target Transactions Between 00:00 to 01:00 on 2026-01-01"
title_in = "Frequency of Multi-Source Transactions Between 00:00 to 01:00 on 2026-01-01"
title_par = "Recurring Transaction Path Frequency Between 00:00 to 01:00 on 2026-01-01"
title_vol = "Distribution of Transaction Sizes \nBetween 00:00 to 01:00 on 2026-01-01"

source60 = BarChartDetail('node','out_degree',xlabel_out,ylabel,
                title_out,"./60mins_out")

target60 = BarChartDetail('node','in_degree',xlabel_in,ylabel,
                         title_in,"./60mins_in")

edge60 = BarChartDetail('source','counts',xlabel_par,ylabel,
                          title_par, "./60mins_par")

vol60 = BarChartDetail(edge_attr,'', 'Trans value - Log scale', ylabel,
                       title_vol, "./60mins_vol")

In [ ]:
# Create stats graphs for 60-minute window
st = pd.to_datetime('2026-01-01 00:00:00')
et = pd.to_datetime('2026-01-02 00:00:00')
windowT = pd.Timedelta(minutes=60)
deltaT = pd.Timedelta(minutes=20)
current_time = st
while current_time <= et - windowT:
    ST = current_time
    ET = current_time + windowT
    ST_str = ST.strftime("%H-%M")
    title_out = f"Frequency of Multi-Target Transactions Between {ST_str} to {(ET).strftime('%H-%M')} on 2026-01-01"
    title_in = f"Frequency of Multi-Source Transactions Between {ST_str} to {(ET).strftime('%H-%M')} on 2026-01-01"
    title_par = f"Recurring Transaction Path Frequency Between {ST_str} to {(ET).strftime('%H-%M')} on 2026-01-01"
    title_vol = f"Distribution of Transaction Sizes \nBetween {ST_str} to {(ET).strftime('%H-%M')} on 2026-01-01"
    mutiTarget60 = BarChartDetail('node','out_degree',xlabel_out,ylabel, title_out,f"./Task3/60mins/{ST_str}_out")
    mutiSource60 = BarChartDetail('node','in_degree',xlabel_in,ylabel, title_in,f"./Task3/60mins/{ST_str}_in")
    edge60 = BarChartDetail('source','counts',xlabel_par,ylabel,title_par, f"./Task3/60mins/{ST_str}_par")
    vol60 = BarChartDetail(edge_attr,'', 'Trans value - Log scale', ylabel, title_vol, f"./Task3/60mins/{ST_str}_vol")
    b60 = B.set_df_window_fromto(ST, ET).process_data(source,target,edge_attr).create_stats_graph(mutiSource60,mutiTarget60,edge60,vol60)
    #b60.create_network_graph()
    current_time += deltaT

In [112]:
from pathlib import Path
from itertools import chain 

files = chain(Path("./Task3/60mins").glob("*_vol.csv"))

def detect_anomalies_zscore_vol(files:chain, value_col:str, z_threshold:int) -> pd.DataFrame:
    '''Detect anomalies using Z-score method'''
    anamaly_reports = [] 
    for file in files:
        df = pd.read_csv(file)
        mean_vol = df[value_col].mean()
        std_vol = df[value_col].std()
        df['z_score'] = (df[value_col] - mean_vol) / std_vol
        anomalies = df[df['z_score'].abs() > z_threshold]
        if not anomalies.empty:
            anomalies['window'] = str(file).split("\\")[-1].replace("_vol.csv","")
            anomalies['mean_volume'] = mean_vol
            anomalies['std_volume'] = std_vol
            anamaly_reports.append(anomalies)
    if anamaly_reports:
        final_report = pd.concat(anamaly_reports, ignore_index=True)
        final_report = final_report.sort_values('z_score', ascending=False).drop_duplicates(subset=['transaction_hash', 'spending_transaction_hash', 'value_spent'])
        final_report.to_csv(f"./Task3/60mins/outputs/anomaly_volume_report_{z_threshold}.csv", index=False)
        print(f"Anomaly report for z-threshold {z_threshold} saved to ./Task3/60mins/outputs/anomaly_volume_report_{z_threshold}.csv")
        return final_report
    else:
        print(f"No anomalies detected for z-threshold {z_threshold}.")

vol_anomalies_5 = detect_anomalies_zscore_vol(files, 'value_spent', z_threshold=5)


C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\464306496.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\464306496.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\464306496.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

Anomaly report for z-threshold 5 saved to ./Task3/60mins/outputs/anomaly_volume_report_5.csv


C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\464306496.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\464306496.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\464306496.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [119]:
chart = (
    ggplot(vol_anomalies_5, aes(x='z_score'))
    + geom_histogram(binwidth=0.5, fill="#5da5da", color="white", alpha=0.8)
    + scale_y_log10()
    + labs(
        title="Distribution of Z-Scores (≥5) for Anomalous Transactions in 60-Minute Rolling Windows (20-Minute Shift) — 2026-01-01",
        x="Z-Score",
        y="Count of Transactions"
    )
    + theme(
            legend_position='right',
            axis_text_x=element_text(rotation=0, ha='center'),
            axis_title=element_text(size=15),
            plot_title=element_text(size=20,hjust=0.5) 
            ) 
)
chart.save("./Task3/60mins/outputs/volume_zscore_distribution.png", dpi=300,limitsize=False)

c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 20 x 10 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/60mins/outputs/volume_zscore_distribution.png
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10


In [ ]:
# Combine all in-degree and out-degree raw csv files for 60-minute windows into a single csv file

# get all relevant csv files
files = chain(
    Path("./Task3/60mins").glob("*_in_raw.csv"),
    Path("./Task3/60mins").glob("*_out_raw.csv"),
)
# read and combine dataframes
combined_df = None
for file in files:
    df = pd.read_csv(file)
    col_name = file.name.replace('_raw.csv','')
    #print(file.name)
    df = df.set_index("node")
    if "in" in col_name:
        df = df[['in_degree']].rename(columns={'in_degree': col_name})
    elif "out" in col_name:
        df = df[['out_degree']].rename(columns={'out_degree': col_name})
    if combined_df is None:
        combined_df = df
    else:
        combined_df = combined_df.join(df, how='outer')
#combined_df = combined_df.fillna(0) # dont fill na with 0, keep it as NaN for missing data 
combined_df.reset_index().to_csv("./Task3/60mins/outputs/combined_in_out_raw.csv", index=False)

In [80]:
# Calculate the difference between consecutive time windows for in-degree and out-degree
combined_df = pd.read_csv("./Task3/60mins/outputs/combined_in_out_raw.csv")
in_cols = sorted([c for c in combined_df.columns if c.endswith('_in')])
out_cols = sorted([c for c in combined_df.columns if c.endswith('_out')])
# split in and out dataframes
df_in = combined_df[in_cols]
df_out = combined_df[out_cols]
# calculate the difference for in-degree, multisource
df_in_diff = df_in.diff(axis=1)
df_in_diff[in_cols[0]] = df_in[in_cols[0]]  
df_in_diff = df_in_diff[in_cols]           
df_in_diff.insert(0, 'node', combined_df['node'])
df_in_diff = df_in_diff.set_index('node')
# calculate the difference for out-degree. multitarget
df_out_diff = df_out.diff(axis=1)
df_out_diff[out_cols[0]] = df_out[out_cols[0]]  
df_out_diff = df_out_diff[out_cols]  
df_out_diff.insert(0, 'node', combined_df['node']) 
df_out_diff = df_out_diff.set_index('node')
# reset index on df_in and df_out
df_in.insert(0, 'node', combined_df['node'])
df_in = df_in.set_index('node')
df_out.insert(0, 'node', combined_df['node'])
df_out = df_out.set_index('node')

#print(df_in_diff)


In [81]:
# Rolling window z-score calculation
def rolling_window_z(df:pd.DataFrame,window:int) -> pd.DataFrame:
    rolling_mean = df.rolling(window=window, axis=1).mean()
    rolling_std = df.rolling(window=window, axis=1).std()
    z = df.sub(rolling_mean).div(rolling_std)
    return z

# Anomaly detection based on z-score
def detect_anomalies(z:pd.DataFrame,z_threshold:float,df:pd.DataFrame,output_path:str) -> pd.DataFrame: # detect anomalies and save to csv
    '''Detect anomalies where the absolute z-score exceeds the given threshold.'''
    anomaly_mask = z.abs() > z_threshold        # identify anomalies where absolute z-score exceeds threshold
    anomalies = df[anomaly_mask].stack().reset_index()
    anomalies.columns = ['node', 'time', 'value']
    anomalies = anomalies.sort_values(by=['time'])
    anomalies.to_csv(output_path, index=False)
    print(f"Anomalies with z-threshold {z_threshold} saved to {output_path}")
    return anomalies

In [82]:
# might not be needed
# Anomaly detection on multisource raw using rolling z-score method
z = rolling_window_z(df_in, window=3) # 20mins window for 60mins data
anomalies_3 = detect_anomalies(z, 3, df_in, "./Task3/60mins/outputs/in_degree_anomalies_3_raw.csv") #3 => 99.73% confidence interval
anomalies_5 = detect_anomalies(z, 5, df_in, "./Task3/60mins/outputs/in_degree_anomalies_5_raw.csv") #5 => 99.99994% confidence interval

C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\1761908318.py:3: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\1761908318.py:4: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead


Anomalies with z-threshold 3 saved to ./Task3/60mins/outputs/in_degree_anomalies_3_raw.csv
Anomalies with z-threshold 5 saved to ./Task3/60mins/outputs/in_degree_anomalies_5_raw.csv


In [83]:
# might not be needed
# Anomaly detection on multitarget raw using rolling z-score method
z = rolling_window_z(df_out, window=3) # 20mins window for 60mins data
anomalies_3 = detect_anomalies(z, 3, df_out, "./Task3/60mins/outputs/out_degree_anomalies_3_raw.csv") #3 => 99.73% confidence interval
anomalies_5 = detect_anomalies(z, 5, df_out, "./Task3/60mins/outputs/out_degree_anomalies_5_raw.csv") #5 => 99.99994% confidence interval

C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\1761908318.py:3: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\1761908318.py:4: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead


Anomalies with z-threshold 3 saved to ./Task3/60mins/outputs/out_degree_anomalies_3_raw.csv
Anomalies with z-threshold 5 saved to ./Task3/60mins/outputs/out_degree_anomalies_5_raw.csv


In [84]:
# Anomaly detection on multisource differences using rolling z-score method 
df_in_diff_abs = df_in_diff.abs()

z = rolling_window_z(df_in_diff_abs, window=3) # 20mins window for 60mins data
anomalies_3 = detect_anomalies(z, 3, df_in_diff_abs, "./Task3/60mins/outputs/in_degree_anomalies_3_diff.csv") #3 => 99.73% confidence interval
anomalies_5 = detect_anomalies(z, 5, df_in_diff_abs, "./Task3/60mins/outputs/in_degree_anomalies_5_diff.csv") #5 => 99.99994% confidence interval

C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\1761908318.py:3: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\1761908318.py:4: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead


Anomalies with z-threshold 3 saved to ./Task3/60mins/outputs/in_degree_anomalies_3_diff.csv
Anomalies with z-threshold 5 saved to ./Task3/60mins/outputs/in_degree_anomalies_5_diff.csv


In [85]:
# Anomaly detection on multitarget differences using rolling z-score method 
df_out_diff_abs = df_out_diff.abs()

z = rolling_window_z(df_out_diff_abs, window=3) # 20mins window for 60mins data
anomalies_3 = detect_anomalies(z, 3, df_out_diff_abs, "./Task3/60mins/outputs/out_degree_anomalies_3_diff.csv") #3 => 99.73% confidence interval
anomalies_5 = detect_anomalies(z, 5, df_out_diff_abs, "./Task3/60mins/outputs/out_degree_anomalies_5_diff.csv") #5 => 99.99994% confidence interval

C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\1761908318.py:3: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
C:\Users\Shenghui\AppData\Local\Temp\ipykernel_14308\1761908318.py:4: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead


Anomalies with z-threshold 3 saved to ./Task3/60mins/outputs/out_degree_anomalies_3_diff.csv
Anomalies with z-threshold 5 saved to ./Task3/60mins/outputs/out_degree_anomalies_5_diff.csv


In [96]:
# Identify nodes with significant changes in multisource differences
threshold = 500
mask = df_in_diff_abs > threshold
result_in = df_in_diff[mask].stack().reset_index()
result_in.columns = ['node', 'time', 'value']
result_in = result_in.sort_values(by=['time'])
unique_nodes = result_in['node'].unique()
print(len(unique_nodes))
df_in_plot = df_in_diff.loc[unique_nodes]
df_in_plot = df_in_plot.reset_index().melt(id_vars='node', var_name='time', value_name='value')

6


In [124]:
# Plot the anomalous nodes with significant changes in multisource differences
colors = [
    "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728",
    "#9467bd", "#8c564b", "#e377c2", "#7f7f7f",
    "#bcbd22", "#17becf", "#aec7e8", "#ffbb78",
    "#98df8a", "#ff9896", "#c5b0d5", "#c49c94",
    "#f7b6d2", "#c7c7c7", "#dbdb8d", "#9edae5"
]

options.figure_size = (20, 10)
p = (ggplot(df_in_plot, aes(x='time', y='value', group='node', color='node')) +
    geom_line() +
    scale_color_manual(values=colors[:len(df_in_plot['node'].unique())]) +
    theme(
            legend_position='right',
            axis_text_x=element_text(rotation=90, ha='center'),
            legend_text=element_text(size=10),
            legend_title=element_text(size=13),
            axis_title=element_text(size=15),
            plot_title=element_text(size=20,hjust=0.5) 
            ) +
    labs(title=f"Anomalous Nodes (Threshold={threshold})",
        x="Time Window",
        y="Diff Value in Satoshi")
    )

p.save(f"./Task3/60mins/outputs/anomalous_nodes_threshold_indiff_{threshold}.png", dpi=300,limitsize=False)

c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 20 x 10 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/60mins/outputs/anomalous_nodes_threshold_indiff_500.png
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\geoms\geom_path.py:100: PlotnineWarning: geom_path: Removed 113 rows containing missing values.


In [88]:
# Identify nodes with significant changes in multitarget differences
threshold = 1000
mask = df_out_diff_abs > threshold
result_out = df_out_diff[mask].stack().reset_index()
result_out.columns = ['node', 'time', 'value']
result_out = result_out.sort_values(by=['time'])
unique_nodes = result_out['node'].unique()
print(len(unique_nodes))

0


In [89]:
# Identify nodes with significant changes in multitarget differences
threshold = 500
mask = df_out_diff_abs > threshold
result_out = df_out_diff[mask].stack().reset_index()
result_out.columns = ['node', 'time', 'value']
result_out = result_out.sort_values(by=['time'])
unique_nodes = result_out['node'].unique()
print(len(unique_nodes))
df_out_plot = df_out_diff.loc[unique_nodes]
df_out_plot = df_out_plot.reset_index().melt(id_vars='node', var_name='time', value_name='value')

1


In [125]:
# Plot the anomalous nodes with significant changes in multitarget differences
options.figure_size = (20, 10)
p = (   ggplot(df_out_plot, aes(x='time', y='value', group='node', color='node')) +
        geom_line() +
        scale_color_manual(values=colors[:len(df_out_plot['node'].unique())]) +
        theme(
                legend_position='right',
                axis_text_x=element_text(rotation=90, ha='center'),
                legend_text=element_text(size=10),
                legend_title=element_text(size=13),
                axis_title=element_text(size=15),
                plot_title=element_text(size=20,hjust=0.5) 
                ) +
        labs(title=f"Anomalous Nodes (Threshold={threshold})",
                x="Time Window",
                y="Diff Value in Satoshi")
)

p.save(f"./Task3/60mins/outputs/anomalous_nodes_threshold_outdiff_{threshold}.png", dpi=300,limitsize=False)

c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 20 x 10 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/60mins/outputs/anomalous_nodes_threshold_outdiff_500.png
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\geoms\geom_path.py:100: PlotnineWarning: geom_path: Removed 15 rows containing missing values.


In [91]:
# For the daily recurring transaction path, calculate the total counts for each node (both source and target)
df_par = pd.read_csv("./Task3/day/daymins_par_raw.csv")

# Flatten 'source' and 'target' into a single Series
all_nodes = df_par[['source', 'target']].stack()  # shape = 2 * n_rows

# Repeat the 'counts' column for both source and target
counts_flat = df_par['counts'].repeat(2).values  # repeat each row's count twice


df_flat = pd.DataFrame({
    'node': all_nodes.values,
    'counts': counts_flat
})

# Group by node and sum counts
df_counts = df_flat.groupby('node', as_index=False)['counts'].sum()
df_counts.to_csv("./Task3/day/daymins_par_node_counts.csv", index=False)


In [92]:
# Summary statistics of node counts on recurring transaction paths
df_counts['counts'].describe()

count    449094.000000
mean          3.414697
std          16.201862
min           1.000000
25%           2.000000
50%           2.000000
75%           3.000000
max        3003.000000
Name: counts, dtype: float64

In [127]:
# Plot top 20 nodes by counts on recurring transaction paths
top20 = df_counts.sort_values('counts', ascending=False).head(20)
p = (
    ggplot(top20, aes(x='reorder(node, counts)', y='counts'))
    + geom_col(fill='#4C72B0')
    + coord_flip()  # horizontal bars (much cleaner)
    + labs(
        title='Top 20 Nodes by Counts on Recurring Transaction Paths on 2026-01-01',
        x='Node',
        y='Counts'
    )
    + theme(
                legend_position='right',
                axis_text_x=element_text(rotation=0, ha='center'),
                axis_title=element_text(size=15),
                plot_title=element_text(size=20,hjust=0.5) 
                )
)
p.save("./Task3/day/top20_nodes_recurring_paths.png", dpi=300,limitsize=False)

c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:623: PlotnineWarning: Saving 20 x 10 in image.
c:\Users\Shenghui\anaconda3\envs\DV\Lib\site-packages\plotnine\ggplot.py:624: PlotnineWarning: Filename: ./Task3/day/top20_nodes_recurring_paths.png
